<a href="https://colab.research.google.com/github/Matadan11/Medicion-Latencia-Homework/blob/main/Tarea3_MPI_Explicada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧪 Tarea 3: Comunicación colectiva y medición de latencia con MPI en Python (Colab)

**Autor:** Daniel Matarrita  
**Curso:** Computación Paralela y Distribuida  
**Profesor:** Johansell Villalobos  

Este notebook implementa y explica paso a paso las dos partes de la Tarea 3:
- Parte A: Operaciones colectivas en MPI (`Bcast`, `Scatter`, `Reduce`)
- Parte B: Medición de latencia punto a punto (`Send`, `Recv`)

> ⚠️ Este notebook está pensado para ejecutarse en **Google Colab**.

In [1]:
# ✅ Paso 1: Instalación de dependencias necesarias en Colab
!apt-get update > /dev/null
!apt-get install -y libopenmpi-dev openmpi-bin > /dev/null
!pip install mpi4py > /dev/null

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


## 🧩 Parte A: Estadísticas globales con operaciones colectivas

En esta parte usaremos `MPI_Bcast`, `MPI_Scatter`, y `MPI_Reduce` para distribuir un arreglo de números y calcular el mínimo, máximo y promedio global.

In [ ]:
%%writefile parteA_estadisticas_mpi.py
from mpi4py import MPI
import numpy as np
import sys

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

# Tamaño del arreglo desde argumentos
N = int(sys.argv[1]) if len(sys.argv) > 1 else 1000000

if N % size != 0:
    if rank == 0:
        print("Error: N no divisible entre procesos")
    exit()

sub_size = N // size
data = None
if rank == 0:
    data = np.random.uniform(0, 100, N)

sub_data = np.empty(sub_size, dtype='d')
comm.Scatter(data, sub_data, root=0)

local_min = np.min(sub_data)
local_max = np.max(sub_data)
local_sum = np.sum(sub_data)

global_min = comm.reduce(local_min, op=MPI.MIN, root=0)
global_max = comm.reduce(local_max, op=MPI.MAX, root=0)
global_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

if rank == 0:
    print(f"Min global: {global_min}")
    print(f"Max global: {global_max}")
    print(f"Promedio global: {global_sum / N}")

In [ ]:
# ▶️ Ejecutar Parte A con 4 procesos
!mpirun -np 4 python3 parteA_estadisticas_mpi.py 1000000

## 📡 Parte B: Medición de latencia punto a punto

Esta parte implementa la medición de latencia entre dos procesos usando `MPI_Send` y `MPI_Recv` de ida y vuelta.

In [ ]:
%%writefile parteB_latencia_mpi.py
from mpi4py import MPI
import numpy as np
import time

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if size != 2:
    if rank == 0:
        print("Este programa requiere exactamente 2 procesos")
    exit()

N = 10000
msg = np.array([0], dtype='b')  # Mensaje de 1 byte

if rank == 0:
    start = MPI.Wtime()
    for _ in range(N):
        comm.Send([msg, MPI.BYTE], dest=1)
        comm.Recv([msg, MPI.BYTE], source=1)
    end = MPI.Wtime()
    total_time = end - start
    avg_latency = (total_time / N) * 1e6  # microsegundos
    print(f"Mensaje de 1 byte transmitido {N} veces.")
    print(f"Latencia promedio (ida y vuelta): {avg_latency:.2f} µs")
    print(f"Latencia estimada unidireccional: {avg_latency/2:.2f} µs")

elif rank == 1:
    for _ in range(N):
        comm.Recv([msg, MPI.BYTE], source=0)
        comm.Send([msg, MPI.BYTE], dest=0)

In [ ]:
# ▶️ Ejecutar Parte B con 2 procesos
!mpirun -np 2 python3 parteB_latencia_mpi.py